## By_product_order_upload

In [46]:
# import library
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
from datetime import datetime
# google_sheet_url = 'https://docs.google.com/spreadsheets/d/1V-ozpnN8LikF2vwgwGu1n6vtHdEZpVpvwxoG6FGq1Zk/export?format=xlsx'
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1V-ozpnN8LikF2vwgwGu1n6vtHdEZpVpvwxoG6FGq1Zk/export?format=xlsx'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_excel(BytesIO(data),sheet_name='11 April').fillna(0)
df.head()
# print(df.dtypes)
df.booth_code = df.booth_code.astype(int)
df.product_qty = df.product_qty.astype(int)
# print(df.dtypes)
df

,date,booth_code,payment_method,product_code,product_qty,counter_name
0,11/04/2022,8123,Credit,PE02,10,By product 1
1,11/04/2022,8123,Credit,PE03,10,By product 1
2,11/04/2022,8123,Credit,GE04,10,By product 1
3,11/04/2022,8115,Credit,PE02,50,By product 1
4,11/04/2022,8115,Credit,PE03,30,By product 1
...,...,...,...,...,...,...
90,11/04/2022,30100,Cash,GE04,2,By product 1
91,11/04/2022,30100,Cash,GE03,5,By product 1
92,11/04/2022,30100,Cash,GE02,5,By product 1
93,11/04/2022,30100,Cash,GE05,2,By product 1


In [47]:
booth_codes = list(set(list(df['booth_code'])))
booth_codes

[99852,
 82190,
 30100,
 99736,
 95386,
 31516,
 8101,
 8105,
 8109,
 8115,
 33080,
 8123,
 82117,
 32716,
 82129,
 99801,
 2913,
 33656,
 82041,
 32890]

In [48]:
def serve_by_product_price_dict(user_profile_id):
    data_dict = {}
    business_obj = Business.objects.get(user_profile_id=user_profile_id)
    business_type_wise_price_obj = BusinessTypeWiseByProductDiscount.objects.filter(business_type_id=business_obj.business_type_id)
    business_type_wise_price_list = list(business_type_wise_price_obj.values_list('id', 'business_type', 'by_product', 'base_price', 'mrp'))
    business_type_wise_price_column = ['id', 'business_type', 'by_product_id', 'base_price', 'mrp']
    business_type_wise_price_df = pd.DataFrame(business_type_wise_price_list, columns=business_type_wise_price_column)
    data_dict['product_price_dict'] = business_type_wise_price_df.groupby('by_product_id').apply(lambda x:x.to_dict('r')[0]).to_dict()
    order_category_dict = {}
    for map_obj in BusinessTypeOrderCategoryeMap.objects.filter(business_type_id=business_obj.business_type_id):
        if not map_obj.order_category.id in order_category_dict:
            order_category_dict[map_obj.order_category.id] = map_obj.payment_option.id
    data_dict['order_category'] = order_category_dict
    return data_dict
def generate_by_product_order_code(date):
    code_bank_obj = BySaleGroupOrderCodeBank.objects.filter()[0]
    current_date = datetime.strptime(date, '%Y-%m-%d')
    prefix_value = str(code_bank_obj.code_prefix) +  str((current_date.year))[2:] + str(current_date.month).zfill(2) + str(current_date.day)
    last_digit = int(code_bank_obj.last_digit)
    new_digit = last_digit + 1
    code_bank_obj.last_digit = new_digit
    code_bank_obj.save()
    last_count = str(new_digit).zfill(4)
    return prefix_value + str(last_count)
def calculate_gst_value(base_price, mrp, qty):
    total_gst = mrp - base_price
    product_cgst_value = total_gst / 2
    product_sgst_value = total_gst / 2
    data_dict = {
        'product_cgst_value': product_cgst_value,
        'product_sgst_value': product_sgst_value
    }
    return data_dict
print('run')

run


In [49]:
for booth_code in booth_codes:
    print(booth_code)
    first_df = df[df['booth_code'] == booth_code].head(1).to_dict('r')[0]
    print(first_df['date'])
    today = datetime.strptime(first_df['date'], '%d/%m/%Y').strftime('%Y-%m-%d')
    print(today)

99852
11/04/2022
2022-04-11
82190
11/04/2022
2022-04-11
30100
11/04/2022
2022-04-11
99736
11/04/2022
2022-04-11
95386
11/04/2022
2022-04-11
31516
11/04/2022
2022-04-11
8101
11/04/2022
2022-04-11
8105
11/04/2022
2022-04-11
8109
11/04/2022
2022-04-11
8115
11/04/2022
2022-04-11
33080
11/04/2022
2022-04-11
8123
11/04/2022
2022-04-11
82117
11/04/2022
2022-04-11
32716
11/04/2022
2022-04-11
82129
11/04/2022
2022-04-11
99801
11/04/2022
2022-04-11
2913
11/04/2022
2022-04-11
33656
11/04/2022
2022-04-11
82041
11/04/2022
2022-04-11
32890
11/04/2022
2022-04-11


<ipython-input-49-dffe5f7b30b4>:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  first_df = df[df['booth_code'] == booth_code].head(1).to_dict('r')[0]


In [50]:
import math
for booth_code in booth_codes:
    print(booth_code)
    
    print('booth_code')
    if BusinessAgentMap.objects.filter(business__code=booth_code).exists():
        first_df = df[df['booth_code'] == booth_code].head(1).to_dict('r')[0]
        
        payment_method_id = BySaleGroupPaymentMethod.objects.get(name=first_df['payment_method']).id
        business_agent_obj = BusinessAgentMap.objects.get(business__code=first_df['booth_code'])
        business_type_id = business_agent_obj.business.business_type.id
        user_profile_id = business_agent_obj.business.user_profile_id
        agent_id = business_agent_obj.agent_id
        agent_user_id = business_agent_obj.business.user_profile.user.id
        business_id = business_agent_obj.business.id
        via_id = 2
        
        print('cross_via')
        today = datetime.strptime(first_df['date'], '%d/%m/%Y').strftime('%Y-%m-%d')
        print('today', today)
        
        data_dict = serve_by_product_price_dict(user_profile_id)
        product_price_dict = data_dict['product_price_dict']
        order_category_dict = data_dict['order_category']
        order_category_id = 2
        
        agent_wallet_obj = ByProductAgentWallet.objects.get(agent_id=agent_id)
        counter_id = None
        is_order_placed = False
        order_code = generate_by_product_order_code(today)
        total_cost = 0
        by_sale_group_obj = BySaleGroup(
            business_id=business_id,
            order_code=order_code,
            zone_id=business_agent_obj.business.zone.id,
            ordered_date=today,
            ordered_via_id=via_id,
            value_before_round_off=0,
            round_off_value=0,
            total_cost=0,
            sale_status_id=1, #ordered
            ordered_by_id=1,
            modified_by_id=1,
            payment_method_id=payment_method_id,
            payment_status_id=1
        )
        by_sale_group_obj.save()
        print('by_sale_group_obj_saved')
        
        super_by_sale_group_obj = BySaleGroupTransactionTrace(
            by_sale_group_id=by_sale_group_obj.id,
            sale_group_order_type_id=1,
        )
        super_by_sale_group_obj.save()
        print('super_by_sale_group_obj_saved')
        
        for index, row in df[df['booth_code'] == booth_code].iterrows():
            product_qty = row['product_qty']
            print(row['product_code'])
            product_obj = ByProduct.objects.get(code=row['product_code'])
            print(row['product_code'])
            current_product_availablity = ByProductCurrentAvailablity.objects.get(by_product_id=product_obj.id)
            if product_qty != 0 and product_qty != None:
                if product_qty <= current_product_availablity.quantity_now:
                    product_base_price = product_qty * product_price_dict[product_obj.id]['base_price']
                    product_mrp = product_qty * product_price_dict[product_obj.id]['mrp']
                    product_gst_dict = calculate_gst_value(product_base_price, product_mrp, product_qty)
                    by_sale_obj = BySale(
                        by_sale_group_id=by_sale_group_obj.id,
                        by_product_id=product_obj.id,
                        count=product_qty,
                        cost=product_base_price,
                        cgst_value=product_gst_dict['product_cgst_value'],
                        sgst_value=product_gst_dict['product_sgst_value'],
                        total_cost=product_mrp,
                        ordered_by_id=1,
                        modified_by_id=1,
                    )
                    by_sale_obj.save()
                    daily_goods_receipt_record_obj = GoodsReceiptRecordForDaily.objects.filter(by_product_id=by_sale_obj.by_product.id, sale_date=today).exclude(quantity_now=0).order_by('id')
                    required_qty = product_qty
                    for goods in daily_goods_receipt_record_obj:
                        if goods.quantity_now >= required_qty:
                            balance_in_goods = goods.quantity_now - Decimal(required_qty)
                            goods.quantity_now = balance_in_goods
                            goods.quantity_now_time = today
                            goods.save()

                            goods_receipt_record_sale_map_obj = DailyGoodsReceiptRecordBySaleMap(
                                daily_goods_receipt_record_id=goods.id,
                                by_sale_id=by_sale_obj.id,
                                quantity_dispatched=required_qty
                            )
                            goods_receipt_record_sale_map_obj.save()
                            break
                        else:
                            required_qty = Decimal(required_qty) - goods.quantity_now
                            goods_receipt_record_sale_map_obj = DailyGoodsReceiptRecordBySaleMap(
                                daily_goods_receipt_record_id=goods.id,
                                by_sale_id=by_sale_obj.id,
                                quantity_dispatched=required_qty
                            )
                            goods_receipt_record_sale_map_obj.save()

                            goods.quantity_now = 0
                            goods.quantity_now_time = today
                            goods.save()


                    if BusinessWiseDailySaleUpdate.objects.filter(sale_date=today, business_id=business_id, by_product_id=product_obj.id).exists():
                        business_sale_obj = BusinessWiseDailySaleUpdate.objects.get(sale_date=today, business_id=business_id, by_product_id=product_obj.id)
                        business_sale_obj.received_quantity = business_sale_obj.received_quantity + product_qty
                        business_sale_obj.save()
                    else:
                        is_yesterday_sale_closed = False
                        if BusinessWiseDailySaleUpdate.objects.filter(sale_date__lt=today, business_id=business_id, by_product_id=product_obj.id).exists():
                            business_sale_obj = BusinessWiseDailySaleUpdate.objects.filter(sale_date__lt=today, business_id=business_id, by_product_id=product_obj.id).order_by('sale_date')[0]
                            if business_sale_obj.closing_quantity is not None:
                                is_yesterday_sale_closed = True
                        else:
                            is_yesterday_sale_closed = True
                        business_sale_obj = BusinessWiseDailySaleUpdate(business_id=business_id,
                                                                       sale_date=today,
                                                                       by_product_id=product_obj.id,
                                                                       received_quantity=product_qty,
                                                                       created_by_id=1,
                                                                       is_yesterday_sale_closed=is_yesterday_sale_closed,
                                                                       updated_by_id=1)
                        business_sale_obj.save()
                    current_product_availablity.quantity_now = current_product_availablity.quantity_now - product_qty
                    current_product_availablity.quantity_now_time = today
                    current_product_availablity.save()
                    total_cost += product_mrp
                    is_order_placed = True
                else:
                    error_dict = {}
                    error_dict['product_name'] = current_product_availablity.by_product.short_name
                    error_dict['required_qty'] = product_qty
                    error_dict['available_qty'] = current_product_availablity.quantity_now
                    print(error_dict)
#                     transaction.savepoint_rollback(sid)
        value_before_round_off = total_cost
        value_after_round_off = math.ceil(value_before_round_off)
        round_off_value = value_after_round_off - value_before_round_off
        saved_by_sale_group_obj = BySaleGroup.objects.get(id=by_sale_group_obj.id)
        saved_by_sale_group_obj.value_before_round_off = value_before_round_off
        saved_by_sale_group_obj.round_off_value = round_off_value
        saved_by_sale_group_obj.total_cost = value_after_round_off
        saved_by_sale_group_obj.save()
        
        employee_id = 34
        counter_id = Counter.objects.get(name=row['counter_name']).id
        if CounterEmployeeTraceMap.objects.filter(employee_id=employee_id, is_active=True,
                                                    collection_date=today).exists():
            counter_employee_trace_obj = \
            CounterEmployeeTraceMap.objects.filter(employee_id=employee_id, is_active=True,
                                                    collection_date=today)[0]
        else:
            counter_employee_trace_obj = CounterEmployeeTraceMap(counter_id=counter_id,
                                             employee_id=employee_id,
                                             is_active=True,
                                             collection_date=today,
                                             start_date_time=today)
            counter_employee_trace_obj.save()
        if CounterEmployeeTraceBySaleGroupMap.objects.filter(
                counter_employee_trace_id=counter_employee_trace_obj.id).exists():
            counter_sale_group_obj = CounterEmployeeTraceBySaleGroupMap.objects.get(
                counter_employee_trace_id=counter_employee_trace_obj.id)
            counter_sale_group_obj.by_sale_group.add(saved_by_sale_group_obj.id)
            counter_sale_group_obj.save()
            print('counter sale updated')
        else:
            counter_sale_group_obj = CounterEmployeeTraceBySaleGroupMap(
                counter_employee_trace_id=counter_employee_trace_obj.id)
            counter_sale_group_obj.save()
            counter_sale_group_obj.by_sale_group.add(saved_by_sale_group_obj.id)
            counter_sale_group_obj.save()
            print('counter sale created')
        if is_order_placed:
            by_sale_obj = BySale.objects.filter(by_sale_group_id=by_sale_group_obj.id)
            by_sale_list = list(by_sale_obj.values_list('id', 'by_product_id', 'count'))
            by_sale_column = ['id', 'product_id', 'count']
            by_sale_df = pd.DataFrame(by_sale_list, columns=by_sale_column)
            if not by_sale_df is None:
                if not super_by_sale_group_obj is None:
                    super_by_sale_group_obj.counter_amount = value_after_round_off
                    super_by_sale_group_obj.counter_id = counter_id
                    super_by_sale_group_obj.order_sale_group_json = by_sale_df.to_dict('r')
                    super_by_sale_group_obj.save()
        if order_category_dict[order_category_id] == 1:
            if value_after_round_off != 0:
                transaction_obj = ByProductTransactionLog(
                    date=today,
                    transacted_by_id=agent_user_id,
                    transacted_via_id=via_id,
                    data_entered_by_id=1,
                    amount=value_after_round_off,
                    transaction_direction_id=1,  # from agent to aavin
                    transaction_mode_id=1,  # Upi
                    transaction_id='1234',
                    transaction_status_id=2,  # completed
                    transaction_approval_status_id=1,  # Accepted
                    transaction_approved_by_id=1,
                    transaction_approved_time=today,
                    wallet_balance_before_this_transaction=agent_wallet_obj.current_balance,
                    wallet_balance_after_transaction_approval=agent_wallet_obj.current_balance,
                    description='Amount for ordering the by product',
                    modified_by_id=1
                )
                transaction_obj.save()
                if not super_by_sale_group_obj is None:
                    super_by_sale_group_obj.counter_transaction_id = transaction_obj.id
                    super_by_sale_group_obj.save()

99852
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
GE03
GE03
counter sale created
82190
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
KI02
KI02
CF01
CF01
VC01
VC01
VC02
VC02
MC01
MC01


<ipython-input-50-5e2515e99ddb>:7: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  first_df = df[df['booth_code'] == booth_code].head(1).to_dict('r')[0]
<ipython-input-48-3a16374cf201>:8: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  data_dict['product_price_dict'] = business_type_wise_price_df.groupby('by_product_id').apply(lambda x:x.to_dict('r')[0]).to_dict()
/home/steve/Software/virtualenv/aavin/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-04-11 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s r

CE01
CE01
MO03
MO03
counter sale updated
30100
booth_code
99736
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
PE02
PE02
PE03
PE03
PE01
PE01
GE04
GE04
GE03
GE03
counter sale updated
95386
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
CF01
CF01
VC01
VC01
VC02
VC02
SC02
SC02
CE01
CE01
CE02
CE02
PC02
PC02
BS01
BS01
MO03
MO03
counter sale updated
31516
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
GE05
GE05
GE04
GE04
counter sale updated
8101
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
PE02
PE02
PE01
PE01
GE04
GE04
GE02
GE02
BA01
BA01
MP01
MP01
PE03
PE03
KI02
KI02
CF01
CF01
VC02
VC02
SC02
SC02
counter sale updated
8105
booth_code
cross_via
today 2022-04-11
by_sale_group_obj_saved
super_by_sale_group_obj_saved
PE02
PE02
PE03
PE03
PE01
PE01
GE02
GE02
MS01
MS01
MP01
MP01
counter sale updated
8109
booth_code
cro

In [10]:
import datetime
close_date = '2022-04-01'

sale_date = str(close_date).split('T')[0]
print(sale_date)
if not DailySaleCloseTrace.objects.filter(sale_date=sale_date).exists():
    daily_sale_record_for_given_date = DailySaleCloseTrace(sale_date=sale_date,
                                                           is_sale_closed=True,
                                                           closed_by_id=18292,
                                                           closed_at=datetime.datetime.now())
    daily_sale_record_for_given_date.save()
else:
    print('else')
    daily_sale_record_for_given_date = DailySaleCloseTrace.objects.get(sale_date=sale_date)
    daily_sale_record_for_given_date.is_sale_closed = True
    daily_sale_record_for_given_date.closed_at = datetime.datetime.now()
    daily_sale_record_for_given_date.save()

date_in_format = datetime.datetime.strptime(str(daily_sale_record_for_given_date.sale_date), "%Y-%m-%d")
print(date_in_format)
next_date = date_in_format + datetime.timedelta(days=1) 
print(next_date)
daily_sale_records = GoodsReceiptRecordForDaily.objects.filter(sale_date=sale_date).exclude(quantity_now=0)

if not DailySaleCloseTrace.objects.filter(sale_date=next_date).exists():
    daily_sale_record_for_given_date = DailySaleCloseTrace(sale_date=next_date,
                                                        is_sale_closed=False)
    daily_sale_record_for_given_date.save()
for daily_sale in daily_sale_records:
    daily_sale_record_obj = GoodsReceiptRecordForDaily(sale_date=next_date,
                                                       by_product_id=daily_sale.by_product_id,
                                                       grn_number=daily_sale.grn_number,
                                                       quantity_at_receipt=daily_sale.quantity_now,
                                                       quantity_now=daily_sale.quantity_now,
                                                       quantity_now_time=datetime.datetime.now(),
                                                       price_per_unit=daily_sale.price_per_unit,
                                                       total_price=daily_sale.total_price,
                                                       )
    daily_sale_record_obj.save()
print('ok')

2022-04-01
2022-04-01 00:00:00
2022-04-02 00:00:00
ok


/home/steve/Software/virtualenv/aavin/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField DailySaleCloseTrace.closed_at received a naive datetime (2022-05-03 05:50:10.732101) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/aavin/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-05-03 05:50:10.747599) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/aavin/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-05-03 05:50:10.750147) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)

In [110]:
daily_sale_records = GoodsReceiptRecordForDaily.objects.filter(sale_date=sale_date).exclude(quantity_now=0)
daily_sale_records.count()

46